# SE4050 - Deep Learning


## Lab Activty 04


###Student ID : IT19132310
### Name : Hettiarachchi L.S

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import matplotlib.pyplot as plt
from IPython.display import HTML
import os
import pathlib
import seaborn as sns

In [ ]:
!unzip /content/Dataset.zip

Archive:  /content/Dataset.zip
  inflating: angry/002_227_543_177_177.png  
  inflating: angry/002_598_301_123_123.png  
  inflating: angry/003_255_980_148_148.png  
  inflating: angry/003_538_101_212_212.png  
  inflating: angry/005_158_215_212_212.png  
  inflating: angry/005_90_152_232_232.png  
  inflating: angry/010_58_283_194_194.png  
  inflating: angry/011_554_840_123_123.png  
  inflating: angry/012_341_666_134_134.png  
  inflating: angry/012_53_622_177_177.png  
  inflating: angry/014_1088_1182_212_212.png  
  inflating: angry/014_124_522_134_134.png  
  inflating: angry/014_532_697_123_123.png  
  inflating: angry/016_308_698_162_162.png  
  inflating: angry/016_565_202_123_123.png  
  inflating: angry/017_306_404_366_366.png  
  inflating: angry/018_474_578_194_194.png  
  inflating: angry/019_109_769_148_148.png  
  inflating: angry/022_179_127_194_194.png  
  inflating: angry/022_914_584_148_148.png  
  inflating: angry/025_306_448_177_177.png  
  inflating: angry/026_54

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = 360
CHANNELS=3
EPOCHS=50

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/dataset/',
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    #class_mode='categorical'
)

Found 455 files belonging to 7 classes.


In [5]:
class_names = dataset.class_names
class_names

['angry', 'crying', 'embarrassed', 'happy', 'pleased', 'sad', 'shock']

In [6]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 360, 360, 3)
[3 5 1 2 4 5 6 2 4 0 2 1 3 3 0 0 5 6 4 6 6 3 6 3 6 2 6 3 1 5 0 6]


In [7]:
len(dataset)

15

In [8]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds) 
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [9]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [10]:
len(train_ds)

12

In [11]:
len(val_ds)

1

In [12]:
len(test_ds)

2

In [13]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [14]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [15]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [16]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [17]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 7

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),

    

])

model.build(input_shape=input_shape)

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 360, 360, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 358, 358, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 179, 179, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 177, 177, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 88, 88, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 86, 86, 64)         

In [19]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [20]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Epoch 1/50
12/12 [==============================] - 100s 8s/step - loss: 1.9313 - accuracy: 0.1643 - val_loss: 1.9128 - val_accuracy: 0.2500
Epoch 2/50
12/12 [==============================] - 89s 7s/step - loss: 1.9269 - accuracy: 0.1755 - val_loss: 1.8474 - val_accuracy: 0.2188
Epoch 3/50
12/12 [==============================] - 82s 7s/step - loss: 1.9114 - accuracy: 0.2201 - val_loss: 1.8639 - val_accuracy: 0.2500
Epoch 4/50
12/12 [==============================] - 81s 7s/step - loss: 1.9023 - accuracy: 0.2228 - val_loss: 1.8322 - val_accuracy: 0.2500
Epoch 5/50
12/12 [==============================] - 82s 7s/step - loss: 1.9109 - accuracy: 0.1699 - val_loss: 1.8653 - val_accuracy: 0.2188
Epoch 6/50
12/12 [==============================] - 81s 7s/step - loss: 1.9097 - accuracy: 0.1894 - val_loss: 1.8656 - val_accuracy: 0.2500
Epoch 6: early stopping


In [ ]:
scores = model.evaluate(test_ds)
scores

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
acc = history.history['accuracy']
print(acc)
print(range(EPOCHS))
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(0, len(history.history['loss'])), acc, label='Training Accuracy')
plt.plot(range(0, len(history.history['loss'])), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(0, len(history.history['loss'])), loss, label='Training Loss')
plt.plot(range(0, len(history.history['loss'])), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
import numpy as np
test_Images = []
test_labels = []
for images_batch, labels_batch in test_ds.take(2):
    
    test_Images.append(images_batch.numpy())
    test_labels.append(labels_batch.numpy())

    first_image = images_batch[6].numpy().astype('uint8')
    first_label = labels_batch[6].numpy()
    
    print("3rd image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[6])])

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[commands != 'README.md']
print('Commands:', commands)

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7, 6))
sns.set(font_scale=1)
sns.heatmap(confusion_mtx,
            xticklabels=commands,
            yticklabels=commands,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.legend(loc=1, prop={'size': 2})
plt.show()

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(20, 20))
for images, labels in test_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4,4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")